## Define variables

In [ ]:
from datetime import datetime

In [ ]:
PROJECT_ID = "rl-llm-dev"
REGION = "us-central1"
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET_NAME = "gs://rl-llm-export-ft"
DOCKER_ARTIFACT_REPO = "llms-on-gke"

## Setting up Artifact Repository

In [ ]:
# Enable the Artifact Registry API service for your project.
! gcloud services enable artifactregistry.googleapis.com

# create a new Docker repository with your region with the description
! gcloud artifacts repositories create {DOCKER_ARTIFACT_REPO} \
    --repository-format=docker \
    --location={REGION} \
    --description="Triton Docker repository"

# verify that your repository was created.
! gcloud artifacts repositories list \
    --location={REGION} \
    --filter="name~"{DOCKER_ARTIFACT_REPO}

! gcloud auth configure-docker {REGION}-docker.pkg.dev --quiet

## Build container

- Build the image and tag the Artifact Registry path that the image will be pushed to

In [ ]:
# JAX to FasterTransformers container image name
JAX_TO_FT_IMAGE_NAME = "jax-to-fastertransformer"
# JAX_TO_FT_IMAGE_URI = f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{DOCKER_ARTIFACT_REPO}/{JAX_TO_FT_IMAGE_NAME}"
JAX_TO_FT_IMAGE_URI = f"gcr.io/{PROJECT_ID}/{DOCKER_ARTIFACT_REPO}/{JAX_TO_FT_IMAGE_NAME}"

In [ ]:
FILE_LOCATION = './src'
! gcloud builds submit \
      --region $REGION \
      --config src/cloudbuild.yaml \
      --substitutions _DOCKERNAME=$JAX_TO_FT_IMAGE_NAME,_IMAGE_URI=$JAX_TO_FT_IMAGE_URI,_FILE_LOCATION=$FILE_LOCATION \
      --timeout "2h" \
      --machine-type=e2-highcpu-32 \
      --quiet